# Laplacian Convolutional Representation (LCR)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(n, tau):
    ell = np.zeros(n)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z, w, lmbda, denominator):
    T = z.shape[0]
    temp = np.fft.fft(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    denominator = lmbda + gamma * np.fft.fft(laplacian(T, tau)) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

dense_y = dense_mat.reshape(-1, order = 'C')
sparse_y = sparse_mat.reshape(-1, order = 'C')
del dense_mat, sparse_mat

import time
start = time.time()
T = sparse_y.shape[0]
lmbda = 5e-6 * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_y, sparse_y, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.02618708671963577
2.4016775552339675

20
0.023773220990012812
2.2383889386305382

30
0.022888036497254603
2.1683646218675143

40
0.02234668627570618
2.1253083707638116

50
0.021973599097177367
2.095785559407199

60
0.021700395092847432
2.0743932680376527

70
0.021491695760635095
2.058253346768129

80
0.02132820031134671
2.045784764909898

90
0.021198286558729466
2.0360176228254843

100
0.021094204200283545
2.0282788598937467

Running time: 2159 seconds.


In [3]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.5
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

dense_y = dense_mat.reshape(-1, order = 'C')
sparse_y = sparse_mat.reshape(-1, order = 'C')
del dense_mat, sparse_mat

import time
start = time.time()
T = sparse_y.shape[0]
lmbda = 5e-6 * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_y, sparse_y, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.033145477900420536
2.763421459707599

20
0.02525152424122118
2.3594170646548904

30
0.02419986536809027
2.2854846035805454

40
0.023613916728243013
2.24168623606139

50
0.02321931671249709
2.2118663848319025

60
0.022932864609828397
2.190299649162898

70
0.02271636209774457
2.1740857270769647

80
0.022547354476308616
2.1615549257673354

90
0.02241387504375854
2.151725250483675

100
0.02230720316469061
2.1439414489054562

Running time: 2120 seconds.


In [4]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.7
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

dense_y = dense_mat.reshape(-1, order = 'C')
sparse_y = sparse_mat.reshape(-1, order = 'C')
del dense_mat, sparse_mat

import time
start = time.time()
T = sparse_y.shape[0]
lmbda = 5e-6 * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_y, sparse_y, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.10253907091756613
6.905642287558284

20
0.03145494332892731
2.7208532023006446

30
0.027103876601495075
2.513602883084324

40
0.026125259848880415
2.46074442221954

50
0.02566411212147537
2.4315526787331745

60
0.025353245030856
2.411106958645215

70
0.025123832850970793
2.3958837263197066

80
0.02494797700152159
2.3841678181505923

90
0.024810131139907404
2.3749578399851816

100
0.024699827742965794
2.367591593623587

Running time: 2108 seconds.


In [5]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

dense_y = dense_mat.reshape(-1, order = 'C')
sparse_y = sparse_mat.reshape(-1, order = 'C')
del dense_mat, sparse_mat

import time
start = time.time()
T = sparse_y.shape[0]
lmbda = 5e-6 * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_y, sparse_y, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.4315617242768932
26.79995272878542

20
0.14775522302879926
9.917739075650493

30
0.0785319079241586
5.330478276189681

40
0.0486372296652227
3.7736340858432484

50
0.039382201687191185
3.30192463442513

60
0.036025620698477125
3.1767402523762343

70
0.034936012665808855
3.1363908387782846

80
0.034483993628456854
3.121504312030326

90
0.03425308775586833
3.113719847697129

100
0.034115541218865474
3.1089444987496275

Running time: 2089 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>